In [38]:
#!/usr/bin/python3

import tensorflow as tf
import numpy as np
import pandas as pd
from keras.models import Model, Sequential
from keras.layers import Input, Dense
import re
import hashlib
import math
import random

from keras import backend as K
import tensorflow as tf

charBeforeAndAfter = 15
modelName = "model_100X50_d_0.h5"

def freeze_session(session, keep_var_names=None, output_names=None, clear_devices=True):
    """
    Freezes the state of a session into a pruned computation graph.

    Creates a new computation graph where variable nodes are replaced by
    constants taking their current value in the session. The new graph will be
    pruned so subgraphs that are not necessary to compute the requested
    outputs are removed.
    @param session The TensorFlow session to be frozen.
    @param keep_var_names A list of variable names that should not be frozen,
                          or None to freeze all the variables in the graph.
    @param output_names Names of the relevant graph outputs.
    @param clear_devices Remove the device directives from the graph for better portability.
    @return The frozen graph definition.
    """
    from tensorflow.python.framework.graph_util import convert_variables_to_constants
    graph = session.graph
    with graph.as_default():
        freeze_var_names = list(set(v.op.name for v in tf.global_variables()).difference(keep_var_names or []))
        output_names = output_names or []
        output_names += [v.op.name for v in tf.global_variables()]
        # Graph -> GraphDef ProtoBuf
        input_graph_def = graph.as_graph_def()
        if clear_devices:
            for node in input_graph_def.node:
                node.device = ""
        frozen_graph = convert_variables_to_constants(session, input_graph_def,
                                                      output_names, freeze_var_names)
        return frozen_graph



np.set_printoptions(linewidth=460)
tf.logging.set_verbosity(tf.logging.INFO)

chars = ' אבגדהוזחטיכלמנסעפצקרשתךםןףץ'

# create labels array (each label is a set of nikuds that should follow latters
nikods = {hashlib.md5("".encode()).hexdigest():""}
f = open("nikud.txt","r")

line = f.readline()
while line:
    line = f.readline()
    line = line.replace('\n', '')
    linep = line.split(':')
    if(len(linep)==2):
        nikods[linep[0]] = linep[1]
f.close()

nikodList = list(nikods)

def sentenceToLabeledData(sentence):
    X = []
    for c in sentence:
        if c in chars:
            Xi = np.zeros((((2*charBeforeAndAfter)+1), len(chars)))
            Xi[0][chars.index(c)] = 1

            for r in range(1,10):
                if len(X)-r > 0:
                    X[len(X)-r][r][chars.index(c)] = 1
                    Xi[10+r] = X[len(X)-r][0]
            X.append(Xi)
    return X

#create model
model = Sequential()

inputS = len(chars)* (2*charBeforeAndAfter+1)
#add model layers
model.add(Dense(100, activation='relu', input_shape=(inputS,)))
model.add(Dense(50, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(len(nikods)))

model.load_weights("weights/"+modelName)

#compile model using mse as a measure of model performance
model.compile(optimizer='adam', loss='mean_squared_error')

#frozen_graph = freeze_session(K.get_session(),
#                              output_names=[out.op.name for out in model.outputs])
# Save to ./model/tf_model.pb
#tf.train.write_graph(frozen_graph, "model", "/www/nikud/tf_model.pb", as_text=False)


sentence = "פעם הוא היה מנקד גם רווחים היום הוא סתם יכול לנקד בלי קשר זה התקדמות"
regex = re.compile('[^'+chars+']')
sentence = regex.sub("", sentence)
X = sentenceToLabeledData(sentence)
X = np.asarray(X)
X = np.reshape(X, (X.shape[0], inputS))
y = model.predict(X)

output = ""
charId = 0

for c in sentence:
    while True:
        maxId = y[charId].argmax(axis=-1)
        oldScore = y[charId][maxId];
        if oldScore <0.5:
            break;
        output += nikods[nikodList[maxId]]
        y[charId][maxId] = 0
    output += c
    charId+=1
print(output)

פעַּם ַהואַ היָה ָמנִקד גם ְרווַּחים ֹהְיום הואּ סתֵםֹ ֵיכוֵל לנּקֹד בָלִי ּקשּר זה ֶׁהְתקדמוִת
